In [62]:
# !pip install -r requirements.txt

In [63]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
# import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [64]:
peak_hours_path = "2022-peak-hours-ca - 2022 Peak Hour Report.csv"
traffic_volumes_path = "2022 AADT DATA - 2022 AADT DATA.csv"

peak_hours_df = pd.read_csv(peak_hours_path)
aadt_df = pd.read_csv(traffic_volumes_path)

peak_hours_df.rename(columns={"RTE": "ROUTE", "CO": "COUNTY"}, inplace=True)

In [65]:
peak_hours_df.columns

Index(['DI', 'ROUTE', 'RTE_SFX', 'COUNTY', 'PM_PFX', 'PM', 'PM_SFX', 'CS',
       'LEG', 'YR', 'AM_DIR', 'AM_WAY_PHV', 'AM_K_FACTOR_AMT',
       'AM_D_FACTOR_AMT', 'AM_KD_FACTOR', 'AM_HOUR', 'AM_DAY', 'AM_MONTH',
       'PM_DIR', 'PM_WAY_PHV', 'PM_K_FACTOR_AMT', 'PM_D_FACTOR_AMT',
       'PM_KD_FACTOR', 'PM_HOUR', 'PM_DAY', 'PM_MONTH'],
      dtype='object')

In [66]:
peak_hours_df[["AM_DAY","PM_DAY","ROUTE","COUNTY","AM_HOUR","PM_HOUR","AM_MONTH","PM_MONTH"]]

,AM_DAY,PM_DAY,ROUTE,COUNTY,AM_HOUR,PM_HOUR,AM_MONTH,PM_MONTH
0,THU,THU,1,ORA,8,16,SEP,SEP
1,TUE,THU,1,SB,6,16,MAY,JUL
2,THU,THU,1,SB,6,17,NOV,NOV
3,SAT,FRI,1,SB,12,15,MAR,JUN
4,TUE,THU,1,SB,7,17,FEB,FEB
...,...,...,...,...,...,...,...,...
1335,TUE,THU,805,SD,8,16,MAY,APR
1336,FRI,THU,805,SD,11,15,JUN,MAY
1337,SAT,SAT,880,ALA,10,13,APR,MAY
1338,WED,FRI,905,SD,7,15,FEB,FEB


In [67]:
# morning_peak_hours_df = 
morning_peak_hours_df = peak_hours_df[["AM_DAY","AM_HOUR","AM_MONTH","AM_DIR","AM_WAY_PHV"]]
afternoon_peak_hours_df = peak_hours_df[["PM_DAY","PM_HOUR","PM_MONTH","PM_DIR","PM_WAY_PHV"]]

In [68]:
morning_peak_hours_df.head()

,AM_DAY,AM_HOUR,AM_MONTH,AM_DIR,AM_WAY_PHV
0,THU,8,SEP,S,2056
1,TUE,6,MAY,S,722
2,THU,6,NOV,S,924
3,SAT,12,MAR,S,523
4,TUE,7,FEB,N,1373


In [69]:
direction_map = {'N': 0, 'S': 1, 'E': 2, 'W': 3}

day_map = {
    'MON': 0, 'TUE': 1, 'WED': 2, 'THU': 3,
    'FRI': 4, 'SAT': 5, 'SUN': 6
}

month_map = {
    'JAN': 0, 'FEB': 1, 'MAR': 2, 'APR': 3,
    'MAY': 4, 'JUN': 5, 'JUL': 6, 'AUG': 7,
    'SEP': 8, 'OCT': 9, 'NOV': 10, 'DEC': 11
}

# encoding functions
def encode_direction(direction):
    return direction_map.get(direction.upper(), -1)

def encode_day(day):
    return day_map.get(day.upper(), -1)

def encode_month(month):
    return month_map.get(month.upper(), -1)

In [70]:
# aadt_df.head()
# aadt_df.isnull().sum()
# aadt_df.shape

# aadt_df.drop(columns=["ROUTE_SFX","PM_SFX","PM_PFX"], inplace=True)
# aadt_df.dropna(subset=['AHEAD_PEAK_HOUR','BACK_PEAK_HOUR'],inplace=True)
# aadt_df['location_string'] = (
#     aadt_df['ROUTE'].astype(str) + ' ' +
#     aadt_df['COUNTY'] + ', ' +
#     aadt_df['LOCATION DESCRIPTION']
# )


In [71]:
# aadt_df['DISTRICT'].unique()

(1820, 14)

In [74]:
# from geopy.geocoders import Nominatim
# from geopy.extra.rate_limiter import RateLimiter

# geolocator = Nominatim(user_agent="traffic_mapper")
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# la_oc_aadt['location'] = la_oc_aadt['location_string'].apply(geocode)
# la_oc_aadt['latitude'] = la_oc_aadt['location'].apply(lambda loc: loc.latitude if loc else None)
# la_oc_aadt['longitude'] = la_oc_aadt['location'].apply(lambda loc: loc.longitude if loc else None)

In [77]:
morning_peak_hours_df.head()

,AM_DAY,AM_HOUR,AM_MONTH,AM_DIR,AM_WAY_PHV
0,THU,8,SEP,S,2056
1,TUE,6,MAY,S,722
2,THU,6,NOV,S,924
3,SAT,12,MAR,S,523
4,TUE,7,FEB,N,1373


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Prepare examples
examples = []
for _, row in morning_peak_hours_df.iterrows():
    am_direction = row.get("AM_DIR")
    am_hour = row.get("AM_HOUR")
    am_day = row.get("AM_DAY")
    am_way_phv = row.get("AM_WAY_PHV")
    am_month = row.get("AM_MONTH")

    # Skip rows with missing critical data
    if any(pd.isna(x) for x in [am_direction, am_hour, am_day, am_way_phv, am_month]):
        continue

    examples.append({
        "AM_HOUR": int(am_hour),
        "AM_DIRECTION": encode_direction(am_direction),
        "AM_DAY": encode_day(am_day),
        "AM_MONTH": encode_month(am_month),
        "AM_WAY_PHV": float(am_way_phv)
    })

print(f"Prepared {len(examples)} examples.")

# Prepare features and target
X = []
y = []

for ex in examples:
    X.append([
        ex["AM_DIRECTION"],
        ex["AM_DAY"],
        ex["AM_MONTH"],
        ex["AM_HOUR"]
    ])
    y.append(ex["AM_WAY_PHV"])  # or normalized PHV multiplier if preferred

X = np.array(X)
y = np.array(y)

# Normalize features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM Regressor
svm = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svm.fit(X_train, y_train)

# Evaluate
score = svm.score(X_test, y_test)
print(f"SVM R² score on test set: {score:.3f}")


Prepared 1340 examples.
SVM R² score on test set: -0.265


In [79]:
print(score)

-0.26481306078949696


## Understanding the AADT Columns

AHEAD means North of a location.
BACK means South of a location. 

These are usually the same because the cars to a location are the same cars coming back from a location. 

- Back Peak Hour - is the estimated volume of traffic going south of a location during RUSH HOUR
- Back Peak MADT - is the estimated MONTHLY volume of traffic going south of a location during RUSH HOUR
- Back Peak AADT - is the estimated ANNUAL volume of traffic going south of a location during RUSH HOUR

In [76]:
la_oc_aadt.columns

Index(['DISTRICT', 'ROUTE', 'ROUTE_SFX', 'COUNTY', 'PM_PFX', 'PM', 'PM_SFX',
       'LOCATION DESCRIPTION', 'BACK_PEAK_HOUR', 'BACK_PEAK_MADT', 'BACK_AADT',
       'AHEAD_PEAK_HOUR', 'AHEAD_PEAK_MADT', 'AHEAD_AADT'],
      dtype='object')